# Internal tflex tool related scripts
> Internal tflex tools

In [2]:
#| default_exp data_processing.tflex_tools

In [1]:
#| hide
%load_ext autoreload
%autoreload 2


In [3]:
#| export
import sys
from pathlib import Path
from fastcore.all import *

In [ ]:
#| export
def generate_tflex_commands(
    input_folder: str,  # Path to folder containing files to process
    python_script: str,  # Path to your Python script
    output_file: str = "tflex_commands.txt",  # Name of tflex command file to create
    file_pattern: str = "*",  # File pattern to match (e.g., "*.txt", "*.csv")
    script_args: str = "",  # Additional arguments for your Python script
    python_executable: str = "python"  # Python executable to use
    )-> str: # Path to generated command file
    """
    Generate tflex command file for parallel processing of files
    
    """
    
    # Get all files matching the pattern
    search_path = Path(input_folder, file_pattern)
    files = Path(search_path).ls()
    
    if not files:
        print(f"No files found matching pattern: {search_path}")
        return None
    
    # Generate commands
    commands = []
    for file_path in files:
        # Create command for each file
        cmd = f"{python_executable} {python_script} {file_path}"
        if script_args:
            cmd += f" {script_args}"
        commands.append(cmd)
    
    # Write commands to file
    with open(output_file, 'w') as f:
        for cmd in commands:
            f.write(cmd + '\n')
    
    print(f"Generated {len(commands)} commands in {output_file}")
    print(f"Files to process: {len(files)}")
    
    return output_file

# Alternative version for more complex scenarios
def generate_tflex_commands_advanced(input_folder, python_script, output_file="tflex_commands.txt",
                                   file_extensions=None, recursive=True, 
                                   max_commands=None, script_template=None):
    """
    Advanced version with more options
    
    Args:
        input_folder (str): Path to folder containing files
        python_script (str): Path to your Python script
        output_file (str): Name of tflex command file
        file_extensions (list): List of file extensions to include ['.txt', '.csv']
        recursive (bool): Search subfolders recursively
        max_commands (int): Limit number of commands (for testing)
        script_template (str): Custom command template with {file} placeholder
    """
    
    input_path = Path(input_folder)
    files = []
    
    # Collect files based on extensions
    if file_extensions:
        for ext in file_extensions:
            if recursive:
                files.extend(input_path.rglob(f"*{ext}"))
            else:
                files.extend(input_path.glob(f"*{ext}"))
    else:
        # Get all files
        if recursive:
            files = [f for f in input_path.rglob("*") if f.is_file()]
        else:
            files = [f for f in input_path.glob("*") if f.is_file()]
    
    # Limit commands if specified
    if max_commands and len(files) > max_commands:
        files = files[:max_commands]
        print(f"Limited to first {max_commands} files")
    
    # Generate commands
    commands = []
    for file_path in files:
        if script_template:
            # Use custom template
            cmd = script_template.format(file=str(file_path))
        else:
            # Default template
            cmd = f"python3 {python_script} {file_path}"
        commands.append(cmd)
    
    # Write to file
    with open(output_file, 'w') as f:
        for cmd in commands:
            f.write(cmd + '\n')
    
    print(f"Generated {len(commands)} commands in {output_file}")
    return output_file

# Example usage functions
def create_data_processing_commands(data_folder, processing_script):
    """Example for data processing workflow"""
    return generate_tflex_commands(
        input_folder=data_folder,
        python_script=processing_script,
        output_file="process_data_commands.txt",
        file_pattern="*.csv",
        script_args="--output-dir ./results"
    )

def create_image_processing_commands(image_folder, processing_script):
    """Example for image processing workflow"""
    return generate_tflex_commands_advanced(
        input_folder=image_folder,
        python_script=processing_script,
        output_file="process_images_commands.txt",
        file_extensions=['.jpg', '.png', '.tiff'],
        recursive=True,
        script_template="python3 {script} --input {file} --output ./processed/"
    )

# Main workflow function
def run_tflex_workflow(input_folder, python_script, tflex_executable="tflex"):
    """
    Complete workflow: generate commands and run tflex
    
    Args:
        input_folder (str): Folder with files to process
        python_script (str): Your Python processing script
        tflex_executable (str): Path to tflex executable
    """
    
    # Generate command file
    cmd_file = generate_tflex_commands(input_folder, python_script)
    
    if cmd_file:
        # Run tflex (you'll need to adjust this based on tflex syntax)
        tflex_cmd = f"{tflex_executable} {cmd_file}"
        print(f"Run this command: {tflex_cmd}")
        
        # Optionally execute automatically
        # import subprocess
        # result = subprocess.run(tflex_cmd, shell=True, capture_output=True, text=True)
        # print(result.stdout)
        
        return tflex_cmd
    
    return None


In [ ]:
import os
import glob
import math
import json
from pathlib import Path

# ===============================================================================
# ORIGINAL FUNCTION - Process each file individually
# ===============================================================================

def generate_tflex_commands(
    input_folder,  # Path to folder containing files to process
    python_script,  # Path to your Python script
    output_file: str = "tflex_commands.txt",  # Name of tflex command file to create
    file_pattern: str = "*",  # File pattern to match (e.g., "*.txt", "*.csv")
    script_args: str = "",  # Additional arguments for your Python script
    python_executable: str = "python3"  # Python executable to use
    ):
    """
    ORIGINAL: Generate tflex command file for parallel processing of individual files
    
    Returns:
        str: Path to generated command file
    """
    
    # Get all files matching the pattern
    search_path = os.path.join(input_folder, file_pattern)
    files = glob.glob(search_path)
    
    if not files:
        print(f"No files found matching pattern: {search_path}")
        return None
    
    # Generate commands - ONE COMMAND PER FILE
    commands = []
    for file_path in files:
        cmd = f"{python_executable} {python_script} {file_path}"
        if script_args:
            cmd += f" {script_args}"
        commands.append(cmd)
    
    # Write commands to file
    with open(output_file, 'w') as f:
        for cmd in commands:
            f.write(cmd + '\n')
    
    print(f"Generated {len(commands)} individual file commands in {output_file}")
    print(f"Files to process: {len(files)}")
    
    return output_file


In [4]:
#| export
# ===============================================================================
# NEW FUNCTION - Process files in batches
# ===============================================================================

def generate_batch_tflex_commands(
    input_folder,  # Path to folder containing files to process
    python_script,  # Path to your Python batch processing script
    files_per_batch: int = 50,  # Number of files to process per batch/job
    output_file: str = "batch_tflex_commands.txt",  # Name of tflex command file to create
    file_pattern: str = "*",  # File pattern to match (e.g., "*.txt", "*.csv")
    file_extensions: list = None,  # List of file extensions ['.txt', '.csv', '.jpg']
    script_args: str = "",  # Additional arguments for your Python script
    python_executable: str = "python3",  # Python executable to use
    recursive: bool = True,  # Search subfolders recursively
    batch_dir: str = "batches"
    )-> dict: # Information about generated batches and commands
    """
    NEW: Generate tflex commands for batch processing - each command processes multiple files
    
    """
    
    input_path = Path(input_folder)
    files = []
    
    # Collect files based on pattern or extensions
    if file_extensions:
        # Use specific extensions
        for ext in file_extensions:
            if recursive:
                files.extend(input_path.rglob(f"*{ext}"))
                files.extend(input_path.rglob(f"*{ext.upper()}"))
            else:
                files.extend(input_path.glob(f"*{ext}"))
                files.extend(input_path.glob(f"*{ext.upper()}"))
    else:
        # Use file pattern
        if recursive:
            files.extend(input_path.rglob(file_pattern))
        else:
            files.extend(input_path.glob(file_pattern))
    
    # Convert to strings
    files = [str(f) for f in files if Path(f).is_file()]
    
    if not files:
        print(f"No files found in {input_folder}")
        return None
    
    # Create batches
    batches = []
    for i in range(0, len(files), files_per_batch):
        batch = files[i:i + files_per_batch]
        batches.append(batch)
    
    print(f"Found {len(files)} files")
    print(f"Created {len(batches)} batches of ~{files_per_batch} files each")
    
    # Create batch directory
    os.makedirs(batch_dir, exist_ok=True)
    
    # Create batch files and commands
    commands = []
    batch_info = []
    
    for batch_idx, batch in enumerate(batches):
        # Create batch file (list of files for this batch)
        batch_filename = f"batch_{batch_idx:04d}.txt"
        batch_filepath = os.path.join(batch_dir, batch_filename)
        
        with open(batch_filepath, 'w') as f:
            for file_path in batch:
                f.write(file_path + '\n')
        
        # Create tflex command for this batch
        cmd = f"{python_executable} {python_script} --batch-file {batch_filepath}"
        if script_args:
            cmd += f" {script_args}"
        
        commands.append(cmd)
        batch_info.append({
            'batch_id': batch_idx,
            'batch_file': batch_filepath,
            'file_count': len(batch),
            'files': batch
        })
    
    # Write commands to tflex file
    with open(output_file, 'w') as f:
        for cmd in commands:
            f.write(cmd + '\n')
    
    # Save batch information
    batch_info_file = os.path.join(batch_dir, "batch_info.json")
    with open(batch_info_file, 'w') as f:
        json.dump(batch_info, f, indent=2)
    
    result = {
        'command_file': output_file,
        'total_files': len(files),
        'total_batches': len(batches),
        'files_per_batch': files_per_batch,
        'batch_directory': batch_dir,
        'batch_info_file': batch_info_file,
        'batches': batch_info
    }
    
    print(f"Generated {len(commands)} batch commands in {output_file}")
    print(f"Batch files saved in: {batch_dir}")
    print(f"Batch info saved in: {batch_info_file}")
    
    return result


In [ ]:
#| export
# ===============================================================================
# UTILITY FUNCTIONS FOR BATCH PROCESSING
# ===============================================================================

def generate_image_batch_commands(
    image_folder,  # Folder containing images
    processing_script,  # Your image processing script
    images_per_batch: int = 100,  # Images per tflex job
    output_dir: str = "./results",  # Output directory
    batch_size: int = 8,  # Batch size for your script's internal processing
    num_processes: int = 4  # Number of processes for multiprocessing
):
    """
    Specialized function for image processing with batches
    """
    
    image_extensions = ['.jpg', '.jpeg', '.png', '.tiff', '.bmp', '.gif']
    
    script_args = f"--output-dir {output_dir} --batch-size {batch_size} --num-processes {num_processes}"
    
    return generate_batch_tflex_commands(
        input_folder=image_folder,
        python_script=processing_script,
        files_per_batch=images_per_batch,
        file_extensions=image_extensions,
        script_args=script_args,
        output_file="image_batch_commands.txt"
    )


In [5]:
#| export
def generate_data_batch_commands(
    data_folder,  # Folder containing data files
    processing_script,  # Your data processing script
    files_per_batch: int = 20,  # Files per tflex job
    output_dir: str = "./processed_data",  # Output directory
    num_processes: int = 8  # Number of processes for multiprocessing
):
    """
    Specialized function for data file processing with batches
    """
    
    data_extensions = ['.csv', '.json', '.txt', '.tsv', '.xlsx']
    
    script_args = f"--output-dir {output_dir} --num-processes {num_processes}"
    
    return generate_batch_tflex_commands(
        input_folder=data_folder,
        python_script=processing_script,
        files_per_batch=files_per_batch,
        file_extensions=data_extensions,
        script_args=script_args,
        output_file="data_batch_commands.txt"
    )


In [ ]:
#| export
# ===============================================================================
# EXAMPLE BATCH PROCESSING SCRIPT TEMPLATE
# ===============================================================================

def create_batch_processing_template(
    template_file="batch_processor_template.py"):
    """
    Create a template for batch processing script that works with the generated commands
    """
    
    template_code = '''import argparse
import multiprocessing as mp
from pathlib import Path
import os

def process_single_file(file_path, output_dir, **kwargs):
    """
    Process a single file - CUSTOMIZE THIS FUNCTION
    
    Args:
        file_path (str): Path to file to process
        output_dir (str): Output directory
        **kwargs: Additional arguments
    """
    
    print(f"Processing: {file_path}")
    
    # YOUR PROCESSING CODE HERE
    # Example:
    # - Load file
    # - Process data/image
    # - Save results
    
    # Example output filename
    input_name = Path(file_path).stem
    output_path = os.path.join(output_dir, f"{input_name}_processed.txt")
    
    # Simulate processing
    with open(output_path, 'w') as f:
        f.write(f"Processed: {file_path}\\n")
    
    return output_path

def process_file_batch(file_paths, output_dir, num_processes=4, **kwargs):
    """
    Process a batch of files using multiprocessing
    
    Args:
        file_paths (list): List of file paths to process
        output_dir (str): Output directory
        num_processes (int): Number of processes to use
        **kwargs: Additional arguments passed to process_single_file
    """
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"Processing {len(file_paths)} files with {num_processes} processes")
    
    # Prepare arguments for multiprocessing
    args_list = [(fp, output_dir) for fp in file_paths]
    
    # Process files in parallel
    with mp.Pool(processes=num_processes) as pool:
        results = pool.starmap(process_single_file, args_list)
    
    successful = sum(1 for r in results if r is not None)
    print(f"Successfully processed {successful}/{len(file_paths)} files")
    
    return results

def main():
    parser = argparse.ArgumentParser(description='Batch file processor')
    parser.add_argument('--batch-file', required=True, 
                       help='File containing list of files to process')
    parser.add_argument('--output-dir', required=True, 
                       help='Output directory')
    parser.add_argument('--num-processes', type=int, default=4,
                       help='Number of processes for multiprocessing')
    parser.add_argument('--batch-size', type=int, default=1,
                       help='Batch size (if needed for your processing)')
    
    args = parser.parse_args()
    
    # Read batch file
    with open(args.batch_file, 'r') as f:
        file_paths = [line.strip() for line in f if line.strip()]
    
    if not file_paths:
        print("No files found in batch file")
        return
    
    # Process the batch
    results = process_file_batch(
        file_paths=file_paths,
        output_dir=args.output_dir,
        num_processes=args.num_processes
    )
    
    print(f"Batch processing completed: {args.batch_file}")

if __name__ == "__main__":
    main()
'''
    
    with open(template_file, 'w') as f:
        f.write(template_code)
    
    print(f"Created batch processing template: {template_file}")
    print("Customize the process_single_file() function for your needs")
    
    return template_file

# ===============================================================================
# EXAMPLE USAGE
# ===============================================================================

if __name__ == "__main__":
    # Example 1: Individual file processing (original method)
    print("=== Original Method: Individual Files ===")
    generate_tflex_commands(
        input_folder="/path/to/data",
        python_script="process_single_file.py",
        file_pattern="*.txt"
    )
    
    print("\\n" + "="*50 + "\\n")
    
    # Example 2: Batch processing (new method)
    print("=== New Method: Batch Processing ===")
    batch_result = generate_batch_tflex_commands(
        input_folder="/path/to/data", 
        python_script="process_batch.py",
        files_per_batch=50,
        file_extensions=['.txt', '.csv', '.json'],
        script_args="--output-dir ./results --num-processes 8"
    )
    
    if batch_result:
        print(f"\\nBatch processing setup complete!")
        print(f"Run: tflex {batch_result['command_file']}")
        print(f"Total batches: {batch_result['total_batches']}")
        print(f"Files per batch: ~{batch_result['files_per_batch']}")
    
    print("\\n" + "="*50 + "\\n")
    
    # Example 3: Create template for batch processor
    print("=== Creating Batch Processor Template ===")
    create_batch_processing_template()
    
    # Example 4: Specialized image processing
    print("\\n=== Image Processing Example ===")
    image_result = generate_image_batch_commands(
        image_folder="/path/to/images",
        processing_script="image_batch_processor.py",
        images_per_batch=100,
        num_processes=4
    )

In [8]:
#| hide
import nbdev; nbdev.nbdev_export('11_data_processing.tflex_tools.ipynb')